In [ ]:
import ee
ee.Initialize()
import pandas as pd

In [ ]:
tile_file = "gee_tile_select.csv"
data = pd.read_csv(tile_file)
minx = data['left']
maxy = data['top']
maxx = data['right']
miny = data['bottom']
tile_id =data["id"]
size_data = len(minx)

In [ ]:
def start_process(i):
    #start data, end date and extent
    s_date = '2015-01-01'
    e_date = '2016-01-01'
    area = ee.Geometry.Rectangle([minx[i], maxy[i], maxx[i], miny[i]])
    tile_name = 'plantation_' + str(tile_id[i])
    
    #entropy
    #Define a neighborhood with a kernel.
    entropy_args = {'radius': 4}
    square = ee.Kernel.square(**entropy_args)
    
    #define image objects
    gain = ee.Image('UMD/hansen/global_forest_change_2017_v1_5').select('gain')
    lossyear = ee.Image('UMD/hansen/global_forest_change_2017_v1_5').select('lossyear')

    
    ##PALSAR
    palsar = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR').filter(ee.Filter.date(s_date, e_date))
    palsar_HH = palsar.select('HH').mean()
    palsar_HV = palsar.select('HV').mean()
    palsar_HV_entropy = palsar_HV.int().entropy(square)

    ##filter using loss year
    zones = gain.eq(1).And((lossyear.lt(12)).Or(lossyear.gt(16)))
    zones = zones.updateMask(zones)

    #reduce raster to polygons
    reduce_combine_args = {
      'reducer2': ee.Reducer.stdDev(),
      'sharedInputs': True
    }
    reducers = ee.Reducer.mean().combine(**reduce_combine_args);
    
    reduce_args = {
      'geometry': area,
      'crs': gain.projection(),
      'scale': 30,
      'geometryType': 'polygon',
      'eightConnected': True,
      'labelProperty': 'regrowth',
      'reducer': reducers,
      'bestEffort': True,
      'maxPixels': 1e18
    }

    #palsar bands
    vectors = zones.addBands(palsar_HH).addBands(palsar_HV).addBands(palsar_HV_entropy).reduceToVectors(**reduce_args)
   
    featureCollection = ee.FeatureCollection([vectors]).flatten()
    
    down_args = {
    'collection': featureCollection,
    'folder': 'plantation_palsar_2015_v3',
    'description': tile_name,
    'fileFormat': 'KML'
    }
    task = ee.batch.Export.table.toDrive(**down_args)
    task.start()

In [1]:
#loop through tiles 10674 processed
for i in range(0,size_data):
    start_process(i)